# Men's team unlikely to medal in floor

## scaling not needed
## Used optimized params


In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from mlxtend.preprocessing import minmax_scaling
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, fbeta_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings



In [2]:
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('../../Combine_Data/men/fx_encoded.csv')
olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_fx.csv')
fxnames = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_fxnames.csv')
fxolymp = pd.read_csv('../../Data/cleandata22-23/men22_23.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538 entries, 0 to 537
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         538 non-null    int64  
 1   D            538 non-null    float64
 2   E            538 non-null    float64
 3   ND           538 non-null    float64
 4   year         538 non-null    int64  
 5   Total        538 non-null    float64
 6   medal        538 non-null    int64  
 7   Name         538 non-null    int64  
 8   Nation       538 non-null    int64  
 9   round_final  538 non-null    bool   
 10  round_qual   538 non-null    bool   
dtypes: bool(2), float64(4), int64(5)
memory usage: 39.0 KB


# Base line using ZeroR

In [5]:

X = df.drop(columns=['medal'])  
y = df['medal']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize  ZeroR classifier
zero_r_clf = DummyClassifier(strategy='most_frequent', random_state=42)

zero_r_clf.fit(X_train, y_train)

# Make predictions on  test data
y_pred = zero_r_clf.predict(X_test)

# Evaluate  model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("ZeroR Classifier")
print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}")


ZeroR Classifier
Accuracy: 0.9722
Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       105
           1       0.00      0.00      0.00         3

    accuracy                           0.97       108
   macro avg       0.49      0.50      0.49       108
weighted avg       0.95      0.97      0.96       108

Confusion Matrix:
[[105   0]
 [  3   0]]


# Model Comparison


In [6]:
y = df['medal']
X = df.drop(columns=['medal'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "SVM": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Network": MLPClassifier(),
}

results = {}

# Iterate through each classifier
for name, clf in classifiers.items():
    # Create a pipeline with StandardScaler for classifiers that require it
    if name in ["SVM", "K-Nearest Neighbors", "Neural Network"]:
        clf = make_pipeline(StandardScaler(), clf)

    # Fit model to training data
    clf.fit(X_train, y_train)

    # Make predictions on test data
    y_pred = clf.predict(X_test)

    # Evaluate model's performance
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    results[name] = {
        "accuracy": accuracy,
        "classification_report": report,
        "confusion_matrix": conf_matrix,
    }

for name, result in results.items():
    print(f"Classifier: {name}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"Classification Report:\n{result['classification_report']}")
    print(f"Confusion Matrix:\n{result['confusion_matrix']}\n")


Classifier: Random Forest
Accuracy: 0.9815
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       105
           1       1.00      0.33      0.50         3

    accuracy                           0.98       108
   macro avg       0.99      0.67      0.75       108
weighted avg       0.98      0.98      0.98       108

Confusion Matrix:
[[105   0]
 [  2   1]]

Classifier: AdaBoost
Accuracy: 0.9815
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       105
           1       1.00      0.33      0.50         3

    accuracy                           0.98       108
   macro avg       0.99      0.67      0.75       108
weighted avg       0.98      0.98      0.98       108

Confusion Matrix:
[[105   0]
 [  2   1]]

Classifier: SVM
Accuracy: 0.9722
Classification Report:
              precision    recall  f1-score   support

           0       0.97 

Classifier: Random Forest
Accuracy: 0.9815

Classifier: AdaBoost
Accuracy: 0.9815

In [7]:
olymp.rename(columns={'D Score': 'D', 'E Score': 'E',
                     'Pen.': 'ND', 'nation': 'Nation'}, inplace=True)

In [8]:
olymp=olymp.drop(columns=['round_TeamFinal', 'round_AAfinal', 'round_TeamQual']) 

In [9]:
desired_column_order = ['Rank', 'D', 'E', 'ND', 'year', 'Total', 'Name', 'Nation',
       'round_final', 'round_qual']

# Create DataFrame with desired column order
olymp = olymp[desired_column_order]

# Grid Search 

In [10]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize AdaBoost classifier
base_classifier = DecisionTreeClassifier(max_depth=3)
clf = AdaBoostClassifier(base_classifier)

# Specify parameter grid for Grid Search
param_grid = {
    'n_estimators': [10, 20, 30],
    'learning_rate': [0.01, 0.1, 1.0]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Fit model to training data using Grid Search
grid_search.fit(X_train, y_train)

# Get best parameters from grid search
best_params = grid_search.best_params_

# Initialize AdaBoost classifier with best parameters
best_clf = AdaBoostClassifier(base_classifier, n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'])

# Fit model to training data
best_clf.fit(X_train, y_train)

# Make predictions on test data
y_pred = best_clf.predict(X_test)

# Evaluate model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("AdaBoost Classifier with Grid Search Results")
print(f"Best Parameters: {best_params}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}")


AdaBoost Classifier with Grid Search Results
Best Parameters: {'learning_rate': 0.1, 'n_estimators': 30}
Accuracy: 0.9907
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       105
           1       0.75      1.00      0.86         3

    accuracy                           0.99       108
   macro avg       0.88      1.00      0.93       108
weighted avg       0.99      0.99      0.99       108

Confusion Matrix:
[[104   1]
 [  0   3]]


# Final hyper-parameter tuning

In [11]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize AdaBoost classifier with best hyperparameters
base_classifier = DecisionTreeClassifier(max_depth=3)
best_clf = AdaBoostClassifier(base_classifier, n_estimators=30, learning_rate=0.1)

# Fit the model to training data
best_clf.fit(X_train, y_train)

# Make predictions on test data
y_pred_prob = best_clf.predict_proba(X_test)[:, 1]

# Adjust the threshold
threshold = 0.5  
y_pred_adjusted = (y_pred_prob > threshold).astype(int)

# Evaluate model's performance with adjusted threshold
accuracy = accuracy_score(y_test, y_pred_adjusted)
report = classification_report(y_test, y_pred_adjusted)
conf_matrix = confusion_matrix(y_test, y_pred_adjusted)

print("AdaBoost Classifier with Best Hyperparameters")
print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}")


AdaBoost Classifier with Best Hyperparameters
Accuracy: 0.9815
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       105
           1       0.67      0.67      0.67         3

    accuracy                           0.98       108
   macro avg       0.83      0.83      0.83       108
weighted avg       0.98      0.98      0.98       108

Confusion Matrix:
[[104   1]
 [  1   2]]


# Model Trained earlier using 2023/2022 data

In [12]:
# Cleaning olymp df

olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_fx.csv')


olymp.rename(columns={'D Score': 'D', 'E Score': 'E',
                     'Pen.': 'ND', 'nation': 'Nation'}, inplace=True)
olymp=olymp.drop(columns=['round_TeamFinal', 'round_AAfinal', 'round_TeamQual']) 
desired_column_order = ['Rank', 'D', 'E', 'ND', 'year', 'Total', 'Name', 'Nation',
       'round_final', 'round_qual'] 

olymp = olymp[desired_column_order]

In [13]:
y_pred = best_clf.predict(olymp)


In [14]:
ypreds  = pd.Series(y_pred)
olymp['ypred']=ypreds


In [15]:
olymp['ypred'].value_counts()

ypred
0    2424
1      15
Name: count, dtype: int64

In [16]:
filtered_df = olymp[olymp['ypred'] == 1]

In [17]:
filtered_df['Name'].unique()

array([ 74, 264, 308, 249, 733, 333, 603,  85])

# fxnames used to match encoded names with actual names

In [18]:
name_encoded_values = [74, 264, 308, 249, 733, 451, 333, 603,  85]

result = fxnames.loc[fxnames['Name_encoded'].isin(name_encoded_values), 'Name']


In [19]:
result.unique()

array(['aurel benovic', 'harry hepworth', 'jake jarman',
       'giarnni regini moran', 'weide su', 'luke whitehouse',
       'jiaxing yang', 'rayderley zapata', 'boheng zhang'], dtype=object)

# fxolymp used to get country info from name list above

In [20]:
fxolymp["Name"]=fxolymp["Name"].str.lower()

In [21]:
names_to_filter = ['aurel benovic', 'harry hepworth', 'jake jarman',
       'giarnni regini moran', 'weide su', 'luke whitehouse',
       'jiaxing yang', 'rayderley zapata', 'boheng zhang']

result = fxolymp[fxolymp['Name'].isin(names_to_filter)]


In [22]:
result['Country'].value_counts()

Country
GBR    237
CHN     77
CRO     41
ESP     27
ENG     23
Name: count, dtype: int64

# Find names from USA

In [23]:
usa = result[result['Country']=='USA']

In [24]:
usa['Name'].value_counts()

Series([], Name: count, dtype: int64)